In [ ]:
import scanpy as sc
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import os
import math
import seaborn as sns
import anndata as ad
import hdf5plugin
import pandas as pd
import sys
import time

In [ ]:
hahn_spatial_assay_anndata = ad.read_h5ad("hahn_spatial_assay_anndata")
hahn_spatial_assay_anndata

In [ ]:
hahn_spatial_assay_anndata.obs['imagerow_int'] = hahn_spatial_assay_anndata.obs['imagerow'] * 100000000
hahn_spatial_assay_anndata.obs['imagecol_int'] = hahn_spatial_assay_anndata.obs['imagecol'] * 100000000

hahn_spatial_assay_anndata.obs['imagerow_int'] = hahn_spatial_assay_anndata.obs['imagerow_int'].astype(int)
hahn_spatial_assay_anndata.obs['imagecol_int'] = hahn_spatial_assay_anndata.obs['imagecol_int'].astype(int)

print(min(hahn_spatial_assay_anndata.obs['imagerow']))
print(max(hahn_spatial_assay_anndata.obs['imagerow']))
print(min(hahn_spatial_assay_anndata.obs['imagecol']))
print(max(hahn_spatial_assay_anndata.obs['imagecol']))
print(min(hahn_spatial_assay_anndata.obs['imagerow_int']))
print(max(hahn_spatial_assay_anndata.obs['imagerow_int']))
print(min(hahn_spatial_assay_anndata.obs['imagecol_int']))
print(max(hahn_spatial_assay_anndata.obs['imagecol_int']))
print()
print(len(hahn_spatial_assay_anndata.obs['imagerow'].value_counts()))
print(len(hahn_spatial_assay_anndata.obs['imagecol'].value_counts()))
print(len(hahn_spatial_assay_anndata.obs['imagerow_int'].value_counts()))
print(len(hahn_spatial_assay_anndata.obs['imagecol_int'].value_counts()))

In [ ]:
num_x_bins = 55
num_y_bins = 55
num_pixel_row = num_x_bins
num_pixel_height = num_y_bins

original_x_max = 51657716610
original_y_max = 51657716610

x_bin_width = original_x_max / num_x_bins
y_bin_width = original_y_max / num_y_bins

# Assuming your DataFrame is called 'df'
hahn_spatial_assay_anndata.obs['pixelrow'] = pd.cut(hahn_spatial_assay_anndata.obs['imagerow_int'], bins=num_x_bins, labels=False, include_lowest=True)
hahn_spatial_assay_anndata.obs['pixelcol'] = pd.cut(hahn_spatial_assay_anndata.obs['imagecol_int'], bins=num_y_bins, labels=False, include_lowest=True)

print(min(hahn_spatial_assay_anndata.obs['imagerow']))
print(max(hahn_spatial_assay_anndata.obs['imagerow']))
print(min(hahn_spatial_assay_anndata.obs['imagecol']))
print(max(hahn_spatial_assay_anndata.obs['imagecol']))
print(min(hahn_spatial_assay_anndata.obs['pixelrow']))
print(max(hahn_spatial_assay_anndata.obs['pixelrow']))
print(min(hahn_spatial_assay_anndata.obs['pixelcol']))
print(max(hahn_spatial_assay_anndata.obs['pixelcol']))
print()
print(len(hahn_spatial_assay_anndata.obs['imagerow'].value_counts()))
print(len(hahn_spatial_assay_anndata.obs['imagecol'].value_counts()))
print(len(hahn_spatial_assay_anndata.obs['pixelrow'].value_counts()))
print(len(hahn_spatial_assay_anndata.obs['pixelcol'].value_counts()))

display(hahn_spatial_assay_anndata.obs)

In [ ]:
# PRE FILTERING - MUST DO TO ENSURE 3 AGES ALL HAVE THE EXACT SAME GENES

hahn_spatial_assay_anndata = hahn_spatial_assay_anndata[hahn_spatial_assay_anndata.obs['slice'] != 'slice1']
hahn_spatial_assay_anndata = hahn_spatial_assay_anndata[hahn_spatial_assay_anndata.obs['slice'] != 'slice1_1_1']
hahn_spatial_assay_anndata = hahn_spatial_assay_anndata[hahn_spatial_assay_anndata.obs['slice'] != 'slice1_2_2']

# Get the indices of genes with non-zero expression in all cells
genes_to_keep = np.where(hahn_spatial_assay_anndata.X.astype(bool).sum(axis=0))[0]

# Filter the AnnData object to retain only the selected genes
hahn_spatial_assay_anndata = hahn_spatial_assay_anndata[:, genes_to_keep]
hahn_spatial_assay_anndata

In [ ]:
hahn_spatial_assay_anndata.obs

In [ ]:
hahn_spatial_assay_anndata.X

In [ ]:
"""
sum_expression = np.sum(np.array(hahn_spatial_assay_anndata.X), axis=1)
plt.hist(sum_expression, bins=10000000)  # Adjust the number of bins as desired
plt.xlabel('Sum of Gene Expression')
plt.ylabel('Frequency')
plt.title('Distribution of Sum of Gene Expression')
plt.xlim(500, 5500)
plt.show()
"""

In [ ]:
def filter_genes_by_sum(ad, threshold):
    # Calculate the sum expression for each gene
    gene_sum = np.array(ad.X.sum(axis=0)).flatten()

    # Create a boolean mask indicating which genes pass the threshold
    pass_threshold = gene_sum >= threshold

    # Filter out the genes using the boolean mask
    filtered_genes = ad[:, pass_threshold]

    return filtered_genes

# Set the threshold for sum expression
threshold = 3900

# Filter out genes based on the sum expression threshold
filtered_ad = filter_genes_by_sum(hahn_spatial_assay_anndata, threshold)

filtered_ad


In [ ]:
filtered_ad = hahn_spatial_assay_anndata

In [ ]:
filtered_ad.var

In [ ]:
start_time = time.time()

# Filter for specific Age and Slice

# age_slice_lst = [('M6', 'slice1'), ('M18', 'slice1_1_1'), ('M21', 'slice1_2_2')]
age_slice_lst = [('M6', 'slice1_3_3'), ('M18', 'slice1_4_4'), ('M21', 'slice1_5_5')]


gene_expression_2D_array_lst = []

gene_lst_for_each_slice_lst = []

for specific_age, specific_slice in age_slice_lst:

    age_specific_anndata = hahn_spatial_assay_anndata[hahn_spatial_assay_anndata.obs['age'] == specific_age]
    specific_age_slice_mouse_anndata = age_specific_anndata[age_specific_anndata.obs['slice'] == specific_slice]

    unique_row_pixel_vals = specific_age_slice_mouse_anndata.obs['pixelrow'].unique()
    unique_row_pixel_vals.sort()
    unique_num_pixel_row = len(unique_row_pixel_vals)

    unique_col_pixel_vals = specific_age_slice_mouse_anndata.obs['pixelcol'].unique()
    unique_col_pixel_vals.sort()
    unique_num_pixel_height = len(unique_col_pixel_vals)

    """
    print("Number of Cells: ", specific_age_slice_mouse_anndata.X.shape[0])
    print("Number of Genes: ", specific_age_slice_mouse_anndata.X.shape[1])
    print("# Pixel for Rows:", num_pixel_row)
    print("# Pixel for Heights:", num_pixel_height)
    print("# Pixels in total:", num_pixel_row*num_pixel_height)
    """

    gene_col_index_lst = list(specific_age_slice_mouse_anndata.var.index)
    gene_col_index_lst.sort()
    
    # for osNMF_genes_only
    """
    OsNMF_gene_list = ['A930009E05Rik', 'Adora2a', 'Amoti1', 'Amotl1', 'Arhgap25', 'Baalc', 'Bend5', 'Boc', 'Box', 'Cadm2', 'Cd4', 'Cplx3', 'Crf1', 'Crlf1', 'Cyp7b1', 'Dact2', 
                 'Ddit4', 'Ddit4l', 'Dnah11', 'E130012A19Rik', 'Eomes', 'Epb4', 'Epb4.1l2', 'Fabp3', 'Fam102b', 'Fam184a', 'Frmd6', 'Gabra6', 'Gjc1', 'Gm10635', 'Gm261', 
                 'Gng4', 'Gpc4', 'Gpo4', 'Gpr6', 'Gprin3', 'Hkdc1', 'Igfn1', 'Ighm', 'Igth1', 'Iqgap2', 'Jam2', 'KIKB', 'Kcnb2', 'Klk8', 'Krt80', 'LOC433727', 'Lct', 'Lef1', 
                 'Lot', 'Mas1', 'Mast', 'Mlip', 'Mp', 'My14', 'Myl4', 'Myo16', 'Nmb', 'Pde1b', 'Pdzm3', 'Pdzrn3', 'Pkp2', 'Plekhg1', 'Plxnd1', 'Pmch', 'Pond1', 'Prox1', 
                 'Rarb', 'Rasa2', 'Rasd2', 'Rg89', 'Rgs16', 'Rgs9', 'Ric8b', 'Satb2', 'Sdk2', 'Serpina9', 'Slitrk6', 'Sp8', 'SpB', 'Spink8', 'Stard8', 'Sy112', 'Synpo2', 
                 'Sytl2', 'Tcf712', 'Tmem196', 'Tmem215', 'Tnip2', 'Tnnc1', 'Tnnt1', 'Tox', 'Trip2', 'Ttc9b', 'Tunc1', 'Unc5d', 'Vangi1', 'Vangl1', 'logap2']
    add_genes = []
    OsNMF_gene_list.extend(add_genes)
    OsNMF_gene_list = list(set(OsNMF_gene_list))
    OsNMF_gene_list.sort()
    #print(gene_list)
    gene_col_index_lst = list(set(gene_col_index_lst) & set(OsNMF_gene_list))
    gene_col_index_lst.sort()
    """
    
    gene_lst_for_each_slice_lst.extend(gene_col_index_lst)
    
    # create an empty 3D numpy array to store the data
    gene_expression_2D_array = np.empty((len(gene_col_index_lst), num_pixel_row, num_pixel_height))
    print(gene_expression_2D_array.shape)

    # define the range of values for each column
    col1_range = range(0, num_pixel_row)
    col2_range = range(0, num_pixel_height)

    # create a new dataframe with all possible combinations of values for the two columns
    all_combin_row_and_col_coord_df = pd.DataFrame([(i, j) for i in col1_range for j in col2_range], columns=['pixelrow', 'pixelcol'])

    index_to_gene_index_dict = {}

    # loop over the genes and fill in the corresponding X values
    count = 0
    for gene_index in gene_col_index_lst:
        index_to_gene_index_dict[count] = gene_index
        gene_pixel_coords_df = specific_age_slice_mouse_anndata[:, gene_index].obs[["pixelrow", "pixelcol"]].copy()
        gene_pixel_coords_df["X_val"] = specific_age_slice_mouse_anndata[gene_pixel_coords_df.index, gene_index].X.flatten().tolist().copy()
        gene_pixel_coords_mean_df = gene_pixel_coords_df.groupby(["pixelrow", "pixelcol"]).mean().reset_index()

        # merge the all_combin_row_and_col_coord_df with the original dataframe and fill missing values with 0 in X_val
        gene_expression_2D_array[count,:,:] = pd.merge(gene_pixel_coords_mean_df, all_combin_row_and_col_coord_df, 
                             on=['pixelrow', 'pixelcol'], how='right').fillna({'X_val': 0}).pivot(index='pixelrow', 
                                                                                                  columns='pixelcol', 
                                                                                                  values='X_val').copy().to_numpy()
        # if count % 10 == 0:
        if count % 1000 == 0:
            print("Finished processing this many genes: ", count)
        count = count + 1

        # Create the directory if it doesn't already exist
        parent_dirname = "REVISED_pixelated_spatial_gene_expression_osNMF_GENE_SLECTION"
        if not os.path.exists(parent_dirname):
            os.makedirs(parent_dirname)

        child_dirname = parent_dirname + '/' + specific_age + "_" + specific_slice + "_dim=" + str(num_x_bins) + "_by_" + str(num_y_bins)
        if not os.path.exists(child_dirname):
            os.makedirs(child_dirname)

        # save the 4-dimensional NumPy array to a NPY file
        np.save(child_dirname + '/' + specific_age + "_" + specific_slice + "_dim=" + str(num_x_bins) + "_by_" + str(num_y_bins) + '_gene_expression_2D_array.npy', gene_expression_2D_array)
        

print()
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: {:.2f} seconds".format(elapsed_time))
print("Elapsed time: {:.2f} minutes".format(elapsed_time/60))
print("Elapsed time: {:.2f} hours".format(elapsed_time/360))

In [ ]:
# check all slices/age have same genes

gene_lst_for_each_slice_lst = list(set(gene_lst_for_each_slice_lst))
print(len(gene_lst_for_each_slice_lst))
gene_lst_for_each_slice_lst.sort()

# Create a dictionary with gene names as keys and their corresponding indices as values
data = {'Gene Name': gene_lst_for_each_slice_lst,
        'Index': range(len(gene_lst_for_each_slice_lst))}

# Create a pandas DataFrame from the dictionary
osNMF_gene_lst_df = pd.DataFrame(data)

# Save the DataFrame to a CSV file named 'gene_data.csv'
osNMF_gene_lst_df.to_csv(parent_dirname + '/' + 'osNMF_gene_lst_df_55_by_55_gene_expression.csv', index=False)


osNMF_gene_lst_df

In [ ]:
M6_slice1_3_3_55_by_55_gene_expression_2D_array = np.load('pixelated_spatial_gene_expression_osNMF_GENE_SLECTION/M6_slice1_3_3_dim=55_by_55/M6_slice1_3_3_dim=55_by_55_gene_expression_2D_array.npy')
M18_slice1_4_4_55_by_55_gene_expression_2D_array = np.load('pixelated_spatial_gene_expression_osNMF_GENE_SLECTION/M18_slice1_4_4_dim=55_by_55/M18_slice1_4_4_dim=55_by_55_gene_expression_2D_array.npy')
M21_slice1_5_5_55_by_55_gene_expression_2D_array = np.load('pixelated_spatial_gene_expression_osNMF_GENE_SLECTION/M21_slice1_5_5_dim=55_by_55/M21_slice1_5_5_dim=55_by_55_gene_expression_2D_array.npy')

print("M6_slice1_3_3_55_by_55_gene_expression_2D_array shape:", M6_slice1_3_3_55_by_55_gene_expression_2D_array.shape)
print("M18_slice1_4_4_55_by_55_gene_expression_2D_array shape:", M18_slice1_4_4_55_by_55_gene_expression_2D_array.shape)
print("M21_slice1_5_5_55_by_55_gene_expression_2D_array shape:", M21_slice1_5_5_55_by_55_gene_expression_2D_array.shape)
print()

# Combine the arrays along the first dimension
combined_mouse_ages_55_by_55_gene_expression = np.concatenate(
    (M6_slice1_3_3_55_by_55_gene_expression_2D_array,
     M18_slice1_4_4_55_by_55_gene_expression_2D_array,
     M21_slice1_5_5_55_by_55_gene_expression_2D_array),
    axis=1
)

# Verify the shape of the combined array
print('combined_mouse_ages_55_by_55_gene_expression shape:', combined_mouse_ages_55_by_55_gene_expression.shape)


# save the 4-dimensional NumPy array to a NPY file
np.save(parent_dirname + '/' + 'combined_mouse_ages_55_by_55_gene_expression.npy', combined_mouse_ages_55_by_55_gene_expression)

In [ ]:
print(gene_lst_for_each_slice_lst)